In [1]:
import random
import re
import os
import uuid
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from duckduckgo_search import DDGS
from dotenv import load_dotenv



load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("API_Key not found. Please set it in your .env file")
print("API key loaded successfully ✅")

## Initialize LLM
llm = ChatOpenAI(
    model = "gpt-5-nano",
    temperature=0.5,
    api_key = api_key
)
print(f"LLM initialized: {llm.model_name}")

API key loaded successfully ✅
LLM initialized: gpt-5-nano


In [3]:
@tool
def weather_tool(city: str) -> str:
    """Returns simulated weather for a given city."""
    city = re.sub(r"[^\w\s]", "", city)

    random.seed(city.lower())
    conditions = ["Sunny", "Cloudy", "Rainy", "Thunderstorm", "Hazy"]

    return (
        f"Weather in {city.title()}:\n"
        f"- Temperature: {random.randint(22, 36)}°C\n"
        f"- Condition: {random.choice(conditions)}\n"
        f"- Humidity: {random.randint(40, 90)}%"
    )


@tool
def dictionary(question: str) -> str:
    """Returns definitions from a simulated dictionary."""
    dictionary_data = {
        "agent": "An entity that perceives its environment and acts upon it.",
        "tool": "A function an agent can use to perform a task.",
        "rag": "Retrieval-Augmented Generation.",
        "llm": "Large Language Model.",
        "embedding": "A numerical representation of text meaning.",
        "ephemeral": "Lasting for a very short time."
    }

    match = re.search(r"define\s+(\w+)", question.lower())
    word = match.group(1) if match else question.lower().strip()

    definition = dictionary_data.get(word)
    if definition:
        return f"{word.title()}: {definition}"
    return f"{word.title()}: Definition not found."


@tool
def web_search(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    results_text = []

    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=3):
            results_text.append(
                f"Title: {r.get('title')}\n"
                f"Summary: {r.get('body')}\n"
                f"Source: {r.get('href')}"
            )

    return "\n---\n".join(results_text) if results_text else "No results found."
print("Retrieval tool created.✅")


Retrieval tool created.✅


In [4]:
tools = [weather_tool, dictionary, web_search]
llm_with_tools = llm.bind_tools(tools)

system_message = SystemMessage(
    content="""
You are a tool-using assistant.

STRICT RULES:
- Use AT MOST ONE tool per user query.
- Use weather_tool ONLY for weather-related questions.
- Use dictionary ONLY when the user asks for a definition.
- Use web_search ONLY when the user explicitly asks to search the web or for news.
- If a question can be answered from general knowledge, DO NOT use any tool.
- NEVER call unnecessary tools.
"""
)

def assistant(state: MessagesState) -> dict:
    messages = [system_message] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState):
    last_message = state["messages"][-1]
    return "tools" if last_message.tool_calls else END


builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", END: END}
)
builder.add_edge("tools", "assistant")

agent = builder.compile(checkpointer=MemorySaver())


In [5]:
def my_agent(user_input: str):
    thread_id = str(uuid.uuid4())  # ✅ NEW thread for every query

    print("\n" + "=" * 70)
    print(f"User: {user_input}")
    print("=" * 70)

    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )

    for msg in result["messages"]:
        if isinstance(msg, AIMessage):
            if msg.tool_calls:
                print(f"\nAgent → Calling tool: {msg.tool_calls[0]['name']}")
            else:
                print(f"\nAgent → {msg.content}")

        elif isinstance(msg, ToolMessage):
            print(f"\nTool Result:\n{msg.content}")

    print("=" * 70 + "\n")



In [6]:
my_agent("What is the weather in Sokoto?")


User: What is the weather in Sokoto?

Agent → Calling tool: weather_tool

Tool Result:
Weather in Sokoto:
- Temperature: 34°C
- Condition: Hazy
- Humidity: 54%

Agent → Current weather in Sokoto: 34°C, hazy, humidity 54%.

If you want, I can pull a broader outlook (wind, precipitation forecast) or convert to Fahrenheit.



In [7]:
my_agent("Define llm")
my_agent("Search for latest AI news")
my_agent("What is the capital of Korea?")


User: Define llm

Agent → Calling tool: dictionary

Tool Result:
Llm: Large Language Model.

Agent → LLM stands for Large Language Model. It is a type of AI model, typically based on transformer architectures, trained on vast amounts of text data to understand and generate human-like language. They are used for tasks such as writing, summarization, translation, question answering, and code generation. Common strengths include fluent, versatile language capabilities; common limitations include the potential to produce incorrect or biased output (hallucinations) and high computational/data requirements. Examples include models like GPT-4, Llama, and Claude.


User: Search for latest AI news


C:\Users\Admin\AppData\Local\Temp\ipykernel_12184\1407580672.py:43: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Agent → Calling tool: web_search

Tool Result:
Title: Artificial intelligence | MIT News | Massachusetts Institute of Technolo…
Summary: Dec 22, 2025 · A “scientific sandbox” lets researchers explore the evolution of vision systems The AI-powered tool could inform the …
Source: https://news.mit.edu/topic/artificial-intelligence2
---
Title: Machine learning | MIT News | Massachusetts Institute of Technolo…
Summary: Dec 18, 2025 · A new way to increase the capabilities of large language models MIT-IBM Watson AI Lab researchers developed an …
Source: https://news.mit.edu/topic/machine-learning
---
Title: Massachusetts Institute of Technology - MIT News
Summary: Dec 19, 2025 · Prof. Daron Acemoglu speaks with New York Times reporter Patricia Cohen about various factors influencing the global …
Source: https://news.mit.edu/topic/artificial-intelligence2?type=2

Agent → Here are some recent AI-news items I found:

- Artificial intelligence | MIT News — Dec 22, 2025
  Title: A “scientific sa